# Custom Training with YOLOv5


* After training is done, the best pretrained weights (best.pt) are downloaded to be tested locally using the offboard ros yolov5 ros node.



# Step 1: Install Requirements

In [2]:
#clone YOLOv5 and install requirements
!git clone https://github.com/ultralytics/yolov5  # clone yolov5 repo

!pip install pillow==10.0.1

%cd yolov5
!sed -i 's/pillow/\# pillow/' requirements.txt
%pip install -qr requirements.txt # install dependencies

import torch
import os
from IPython.display import Image, clear_output  # to display images


# To use the free GPU by google colab
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")


Cloning into 'yolov5'...
remote: Enumerating objects: 16562, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 16562 (delta 20), reused 30 (delta 13), pack-reused 16522
Receiving objects: 100% (16562/16562), 15.08 MiB | 24.71 MiB/s, done.
Resolving deltas: 100% (11377/11377), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 16.0 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0


/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.8/750.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00
Setup complete. Using torch 2.2.1+cu121 (Tesla T4)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Step 2: Import Dataset


In [4]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/drive/MyDrive/Spring_2024/Deep_Learning"

# Step 3: Train Custom YOLOv5 model

Options for training are as follows:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Here we used only 100 epochs!)
- **data:** dataset locaiton is saved in the `dataset.location`
- **weights:** specify a path to weights to start transfer learning from. Here we choose the yolov5s pretrained checkpoint, which is the second smallest and fastest model from yolov5.
- **cache:** cache images for faster training

In [5]:
!python train.py --img 640 --batch 16 --epochs 30 --data /content/drive/MyDrive/Spring_2024/Deep_Learning/dataset.yaml --weights yolov5s.pt --cache
#!python train.py --img 640 --epochs 3 --data coco128.yaml --weights yolov5s.pt

2024-04-20 23:50:57.934420: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 23:50:57.934468: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-20 23:50:57.935861: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/Spring_2024/Deep_Learning/dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, 

#Run Inference  With Trained Weights


In [6]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.1 --source /content/drive/MyDrive/Spring_2024/Deep_Learning/images

detect: weights=['runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/Spring_2024/Deep_Learning/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-304-g22361691 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/70 /content/drive/MyDrive/Spring_2024/Deep_Learning/images/frame000000.png: 384x640 1 apriltag, 50.0ms
image 2/70 /content/drive/MyDrive/Spring_2024/Deep_Learning/images/frame000001.png: 384x640 1 apriltag, 7.4ms
image 3/70 /content/drive/MyDrive/Spring_2024/Deep_Learning/images/frame0

In [7]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.png'):
    display(Image(filename=imageName))
    print("\n")

Output hidden; open in https://colab.research.google.com to view.

# Download best weights in pytorch format and future exports in different model formats:



In [8]:
#export model's weights for future use in offboard applications
from google.colab import files
files.download('/content/yolov5/runs/train/exp/weights/best.pt') # change the exp number if you ran multiple training sessions

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
# Convert the best weights we have to .pb model to be then converted to .tflite
# with the correct parameters using modalai structure

!python export.py --weights runs/train/exp/weights/best.pt --include pb --img 416 --data /content/drive/MyDrive/Spring_2024/Deep_Learning/dataset.yaml

export: data=/content/drive/MyDrive/Spring_2024/Deep_Learning/dataset.yaml, weights=['runs/train/exp/weights/best.pt'], imgsz=[416], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['pb']
YOLOv5 🚀 v7.0-304-g22361691 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs

PyTorch: starting from runs/train/exp/weights/best.pt with output shape (1, 10647, 6) (13.7 MB)
2024-04-20 23:54:16.609149: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 23:54:16.609209: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register

In [10]:
# Convert the best weights we have to fp16 Tflite model to be deployed on voxl
# change the exp number if you ran multiple training sessions
!python export.py --weights runs/train/exp/weights/best.pt --include tflite --img 416 --data /content/drive/MyDrive/Spring_2024/Deep_Learning/dataset.yaml

export: data=/content/drive/MyDrive/Spring_2024/Deep_Learning/dataset.yaml, weights=['runs/train/exp/weights/best.pt'], imgsz=[416], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['tflite']
YOLOv5 🚀 v7.0-304-g22361691 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs

PyTorch: starting from runs/train/exp/weights/best.pt with output shape (1, 10647, 6) (13.7 MB)
2024-04-20 23:54:43.936275: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 23:54:43.936337: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to regi

In [11]:
#export model's weights for future use in offboard applications
from google.colab import files
files.download('/content/yolov5/runs/train/exp/weights/best-fp16.tflite') # change the exp number if you ran multiple training sessions
files.download('/content/yolov5/runs/train/exp/weights/best.pb') # change the exp number if you ran multiple training sessions

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
# Check the input and output layer names for model conversion

import tensorflow as tf
from tensorflow.python.platform import gfile

# Load the PB model.
with tf.io.gfile.GFile('runs/train/exp/weights/best.pb', 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

# Print all the tensor names.
for node in graph_def.node:
    print(node.name)


x
model/tfc3_5/tf_conv_42/beta
model/tfc3_5/sequential_10/tf_bottleneck_8/tf_conv_44/beta
model/tfc3_5/sequential_10/tf_bottleneck_8/tf_conv_43/beta
model/tfc3_5/tf_conv_40/beta
model/tf_conv_39/beta
model/tfc3_4/tf_conv_36/beta
model/tfc3_4/sequential_9/tf_bottleneck_7/tf_conv_38/beta
model/tfc3_4/sequential_9/tf_bottleneck_7/tf_conv_37/beta
model/tfc3_4/tf_conv_34/beta
model/tf_conv_33/beta
model/tfsppf/tf_conv_32/beta
model/tfsppf/tf_conv_31/beta
model/tfc3_3/tf_conv_28/beta
model/tfc3_3/sequential_8/tf_bottleneck_6/tf_conv_30/beta
model/tfc3_3/sequential_8/tf_bottleneck_6/tf_conv_29/beta
model/tfc3_3/tf_conv_26/beta
model/tf_conv_25/beta
model/tfc3_2/tf_conv_18/beta
model/tfc3_2/sequential_6/tf_bottleneck_5/tf_conv_24/beta
model/tfc3_2/sequential_6/tf_bottleneck_5/tf_conv_23/beta
model/tfc3_2/sequential_6/tf_bottleneck_4/tf_conv_22/beta
model/tfc3_2/sequential_6/tf_bottleneck_4/tf_conv_21/beta
model/tfc3_2/sequential_6/tf_bottleneck_3/tf_conv_20/beta
model/tfc3_2/sequential_6/tf_bo